In [3]:
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [4]:
from glove import Corpus, Glove

Утилсы: регулярки, загрузка файлов и т.д.

In [5]:
from re import sub, compile

def cut(data):
    r = compile(r'<.*?>|>>.*|&#(47|92);|&quot;|&gt;|(http|https):.*')
    return r.sub('', data)

def punctuate(data):
    r = compile( r'([a-zA-Z])([,.!])', r'\1 \2')
    return r.sub('', data)

def pos_cut(s):
    s = sub(r"ADJF", "ADJ", s)
    s = sub(r"ADVB", "ADV", s)
    s = sub(r"INFN", "VERB", s)
    return s

def topic_cut(word):
    return sub('[^a-zA-Zа-яА-Я]', '', word)

def remove_noise(s):
    remove_noise = compile('(@[A-Za-z_0-9]+)|\[.+]\,|\[.+]|([^0-9A-Za-zА-Яа-я[\.’,?!-:] \t])'
                           '|(\w+:\/\/\S+)')
    return (sub(remove_noise, '',  s))

def check_for_rus_lan(word):
    regex = compile('[А-Яа-я]+$')
    if regex.match(word):
        return True
    return False

In [6]:
BOARD = 'b'

In [7]:
from pickle import load

documents = load(open('pickle/' + BOARD + '.pickle', 'rb'))

In [ ]:
thefile = open('w2.txt', 'w')

for item in documents:
    thefile.write("%s\n" % item)

Инициализация стоп-слов

In [8]:
from nltk.corpus import stopwords

stop = stopwords.words('russian')
stop += stopwords.words('english')

my_stop = ['весь', 'всякий', 'тот', 'это', 'самое', 'каким', 'сама', 'никак', 'она', 'она', 'оно', 'какие', 'какого', 'которая', 'многое', 'чему', 'всему', 'раза', 'сразу', 'весь', 'раз', 'пор', 'например', 'вроде', 'которые', 'который', 'просто', 'очень', 'почему', 'вообще', 'ещё', 'типа', 'ради', 'всё', 'хотя', 'тебе', 'нужно', 'пока']
my_stop_verb = ['сделать', 'можешь', 'могу', 'могут', 'делать', 'будешь', 'быть', 'будут', 'смочь']

[stop.append(morph.parse(i)[0].normal_form) for i in my_stop]
[stop.append(morph.parse(i)[0].normal_form) for i in my_stop_verb]

dvach_stop = ['такое', 'лол', 'бля']

stop += dvach_stop

Загрузка токенизатора, токенизация данных

In [9]:
alpha_tokenizer = RegexpTokenizer('[A-Za-zА-Яа-я]\w+')

def my_tokenize(sent):
    sents = list(alpha_tokenizer.tokenize(sent))
    return list([morph.parse(sent.lower())[0].normal_form for sent in sents if sent.lower() not in stop and len(sent.lower()) > 1])

model_data = [my_tokenize(sent) for sent in documents]

In [10]:
from os import path, makedirs
from pickle import dump

subdir = 'pickle'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, 'model_data.pickle')

with open(file_path, 'wb') as f:
    dump(documents, f)

Обучение Word2Vec-модели

In [9]:
corpus = Corpus()
corpus.fit(model_data, window=3)

In [10]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [11]:
glove.add_dictionary(corpus.dictionary)

Сериализация модели

In [12]:
from os import path, makedirs
from pickle import dump

subdir = 'models'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, BOARD + '_glove.pickle')

with open(file_path, 'wb') as f:
    dump(glove, f)

In [13]:
glove.most_similar(morph.parse('абу')[0].normal_form)

[('она', 0.74974750400770906),
 ('это', 0.74152926959798227),
 ('сиська', 0.73945613742605198),
 ('любовь', 0.73912827160230721)]